<a href="https://colab.research.google.com/github/shumph10/Housing_Assistance_Analysis/blob/sarah_preprocessing/Sup_ML_Modelipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
import keras_tuner
from tensorflow import keras
import kerastuner as kt
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
rent_df = pd.read_csv("sample_data/Sec8_FMR_17&22.csv")
rent_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  if __name__ == '__main__':


,County_Name,ELI_2_17,ELI_2_22,Metro_Area_Name,State_Alpha,_id,county,county_town_name,fmr0_17,fmr1_17,...,l50_2_17,l50_2_22,l80_2_17,l80_2_22,median2017,median2022,metro,metro_code,state,state_name
0,Autauga County,16240,18310,"Montgomery, AL MSA",AL,100199999,1,Autauga County,587,682,...,23900,30200,38200,48350,59700,75500,1,METRO33860M33860,1,Alabama
1,Baldwin County,16240,20550,"Daphne-Fairhope-Foley, AL MSA",AL,100399999,3,Baldwin County,807,847,...,24600,34200,39400,54750,61500,85500,1,METRO19300M19300,1,Alabama
2,Barbour County,16240,18310,"Barbour County, AL",AL,100599999,5,Barbour County,501,505,...,18450,24650,29550,39450,46100,46400,0,NCNTY01005N01005,1,Alabama
3,Bibb County,16240,20400,"Birmingham-Hoover, AL HUD Metro FMR Area",AL,100799999,7,Bibb County,665,751,...,25250,33950,40400,54300,63100,84800,1,METRO13820M13820,1,Alabama
4,Blount County,16240,20400,"Birmingham-Hoover, AL HUD Metro FMR Area",AL,100999999,9,Blount County,665,751,...,25250,33950,40400,54300,63100,84800,1,METRO13820M13820,1,Alabama


In [3]:
rent_df.columns

Index(['County_Name', 'ELI_2_17', 'ELI_2_22', 'Metro_Area_Name', 'State_Alpha',
       '_id', 'county', 'county_town_name', 'fmr0_17', 'fmr1_17', 'fmr2_17',
       'fmr3_17', 'fmr4_17', 'fmr_0_22', 'fmr_1_22', 'fmr_2_22', 'fmr_3_22',
       'fmr_4_22', 'l50_2_17', 'l50_2_22', 'l80_2_17', 'l80_2_22',
       'median2017', 'median2022', 'metro', 'metro_code', 'state',
       'state_name'],
      dtype='object')

In [4]:
# Drop the non-beneficial ID columns
sec8_df = rent_df.copy()
sec8_df.drop(columns=["County_Name", "Metro_Area_Name","State_Alpha", "_id", "county_town_name","metro_code","state_name"], inplace=True)
sec8_df.head()

,ELI_2_17,ELI_2_22,county,fmr0_17,fmr1_17,fmr2_17,fmr3_17,fmr4_17,fmr_0_22,fmr_1_22,...,fmr_3_22,fmr_4_22,l50_2_17,l50_2_22,l80_2_17,l80_2_22,median2017,median2022,metro,state
0,16240,18310,1,587,682,822,1054,1425,643,764,...,1156,1494,23900,30200,38200,48350,59700,75500,1,1
1,16240,20550,3,807,847,977,1422,1634,772,777,...,1348,1715,24600,34200,39400,54750,61500,85500,1,1
2,16240,18310,5,501,505,671,839,958,532,536,...,871,980,18450,24650,29550,39450,46100,46400,0,1
3,16240,20400,7,665,751,866,1163,1298,765,820,...,1220,1316,25250,33950,40400,54300,63100,84800,1,1
4,16240,20400,9,665,751,866,1163,1298,765,820,...,1220,1316,25250,33950,40400,54300,63100,84800,1,1


In [5]:
#check data types are able to be input into a machine learning model
sec8_df.dtypes

ELI_2_17      int64
ELI_2_22      int64
county        int64
fmr0_17       int64
fmr1_17       int64
fmr2_17       int64
fmr3_17       int64
fmr4_17       int64
fmr_0_22      int64
fmr_1_22      int64
fmr_2_22      int64
fmr_3_22      int64
fmr_4_22      int64
l50_2_17      int64
l50_2_22      int64
l80_2_17      int64
l80_2_22      int64
median2017    int64
median2022    int64
metro         int64
state         int64
dtype: object

In [6]:
#check for null values
sec8_df.isnull().sum(axis=0)

ELI_2_17      0
ELI_2_22      0
county        0
fmr0_17       0
fmr1_17       0
fmr2_17       0
fmr3_17       0
fmr4_17       0
fmr_0_22      0
fmr_1_22      0
fmr_2_22      0
fmr_3_22      0
fmr_4_22      0
l50_2_17      0
l50_2_22      0
l80_2_17      0
l80_2_22      0
median2017    0
median2022    0
metro         0
state         0
dtype: int64

In [7]:
#get number of unique categories in columns
sec8_df.nunique()

ELI_2_17      148
ELI_2_22      206
county        327
fmr0_17       450
fmr1_17       479
fmr2_17       507
fmr3_17       678
fmr4_17       763
fmr_0_22      537
fmr_1_22      555
fmr_2_22      589
fmr_3_22      773
fmr_4_22      864
l50_2_17      364
l50_2_22      421
l80_2_17      372
l80_2_22      446
median2017    571
median2022    688
metro           2
state          56
dtype: int64

In [8]:
sec8_df.describe()

,ELI_2_17,ELI_2_22,county,fmr0_17,fmr1_17,fmr2_17,fmr3_17,fmr4_17,fmr_0_22,fmr_1_22,...,fmr_3_22,fmr_4_22,l50_2_17,l50_2_22,l80_2_17,l80_2_22,median2017,median2022,metro,state
count,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,...,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000,4756.000000
mean,17343.048780,20855.271236,74.399916,613.288267,683.081581,853.594828,1127.046257,1289.969722,723.728343,793.328217,...,1287.335786,1480.903280,26377.176198,33427.985702,41635.786375,53087.657696,62886.795627,80179.878049,0.421362,30.377839
std,2765.370976,4306.183160,99.185176,182.971476,213.652341,261.968694,336.462556,394.393776,256.193453,286.709824,...,432.989327,502.595689,6160.930290,8192.776692,8706.255703,12320.645352,16976.552090,22665.600818,0.493829,15.046608
min,5800.000000,5950.000000,1.000000,323.000000,325.000000,432.000000,540.000000,595.000000,356.000000,370.000000,...,549.000000,574.000000,9600.000000,9850.000000,15400.000000,15800.000000,15800.000000,21600.000000,0.000000,1.000000
25%,16240.000000,18310.000000,13.000000,486.000000,538.000000,681.000000,895.000000,1014.000000,555.000000,600.000000,...,1000.000000,1122.000000,22400.000000,28600.000000,35800.000000,45800.000000,51800.000000,64800.000000,0.000000,21.000000
50%,16240.000000,19250.000000,37.000000,563.000000,606.000000,758.000000,1010.500000,1182.000000,639.000000,698.000000,...,1129.500000,1313.000000,25200.000000,32100.000000,40350.000000,51350.000000,61200.000000,77900.000000,0.000000,27.000000
75%,17550.000000,22600.000000,105.000000,700.000000,776.250000,976.000000,1274.000000,1471.000000,810.000000,884.000000,...,1472.000000,1727.000000,29200.000000,37650.000000,46750.000000,60250.000000,72000.000000,91400.000000,1.000000,42.000000
max,33700.000000,44750.000000,999.000000,1915.000000,2411.000000,3018.000000,3927.000000,4829.000000,2145.000000,2631.000000,...,4111.000000,4473.000000,56200.000000,74600.000000,84300.000000,119300.000000,142800.000000,180900.000000,1.000000,78.000000


In [9]:
#split preprocessed data into features and target arrays
y = sec8_df.metro
X = sec8_df.drop("metro", 1)

#split preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0, stratify=y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
#Make sure scaled correctly
X_train.shape

(3567, 20)

In [12]:
X_train_scaled.shape

(3567, 20)

In [13]:
#define the input features for the model
number_input_features = len(X_train.iloc[0])
number_input_features

20

In [14]:
#Create a function to create a new Sequential model with hyperparameter(hp) options
def build_model(hp):
  model = keras.Sequential()
  #allow kerastuner to decide which activation function to use in hidden layers
  activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])


  #allow kerastuner to decide number of neurons in first layer
  model.add(keras.layers.Dense(units=hp.Int('first_units', 
                                            min_value=1,
                                            max_value=10,
                                            step=2), activation=activation, input_dim=number_input_features))
  #allow kerastuner to decide number of hidden layers and neurons in hidden layers
    #using max 2 hidden layers since 3 hidden layers are suspected to be able
    #to appropriately handle any ml model in theory
  for i in range(hp.Int('layers', 1, 2)):
    model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                 min_value=1,
                                                 max_value=10,
                                                 step=2), activation=activation))
    
  model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  #compile the model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [15]:
#use hyperband (optimized version of random search) to 
  #use early stopping to speed up the hp tuning process
Tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=20,
                     hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [16]:
#run kerastuner to find the hp's
Tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 180 Complete [00h 00m 21s]
val_accuracy: 0.8477712273597717

Best val_accuracy So Far: 0.9007569551467896
Total elapsed time: 00h 18m 04s
INFO:tensorflow:Oracle triggered exit


In [23]:
#Get the best model
best_hyper = Tuner.get_best_hyperparameters(1)[0]
best_hyper

In [18]:
best_hyper.values

{'activation': 'tanh',
 'first_units': 5,
 'layers': 6,
 'tuner/bracket': 0,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_0': 3,
 'units_1': 7,
 'units_2': 9,
 'units_3': 1,
 'units_4': 3,
 'units_5': 7}

In [24]:
Tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
activation: tanh
first_units: 5
layers: 6
units_0: 3
units_1: 7
units_2: 9
units_3: 1
units_4: 3
units_5: 7
tuner/epochs: 50
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9007569551467896
Trial summary
Hyperparameters:
activation: tanh
first_units: 9
layers: 3
units_0: 7
units_1: 7
units_2: 9
units_3: 7
units_4: 7
units_5: 3
tuner/epochs: 50
tuner/initial_epoch: 17
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0137
Score: 0.8965517282485962
Trial summary
Hyperparameters:
activation: relu
first_units: 9
layers: 5
units_0: 3
units_1: 5
units_2: 5
units_3: 3
units_4: 1
units_5: 7
tuner/epochs: 50
tuner/initial_epoch: 17
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0165
Score: 0.8948696255683899
Trial summary
Hyperparameters:
activation: tanh
first_units: 7
layers: 6
units_0: 3
units_1: 3
units_2: 5
units_3: 9
units_4: 9
units_5: 9
tuner/epochs: 50
tuner/initial_epoch:

In [21]:
best_model = Tuner.get_best_models(1)[0]


In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

38/38 - 1s - loss: 0.3081 - accuracy: 0.9008 - 676ms/epoch - 18ms/step
Loss: 0.3080722689628601, Accuracy: 0.9007569551467896


In [28]:
#save to a hdf5 file
best_model.save("trained_application_nn.h5")

#import model to new obj
nn_imported = tf.keras.models.load_model('sample_data/trained_application_nn.h5')

OSError: ignored